# Machine Learning prediction of heatwaves
**Alessandro Lovo and George Miloshevich**

*To see equations in this file you may need Markdown Math extension on your vs code for example...*

Below we will need some standard routines but we also introduce utilities.py that allows more efficient processing of dictionaries and json files for example.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
from pathlib import Path

import utilities as ut

# Data

## Fetch the data

The first procedure involves downloading the data to the local space which we have hosted on owncloud

In [2]:
data_info_dict = ut.json2dict('data_info.json')

def fetch(name, destination='./data', reload=False):
    destination = Path(destination).resolve()
    if not os.path.exists(destination):
        destination.mkdir(parents=True)

    if name.endswith('.nc'):
        name = name[:-3]

    file = destination / f'{name}.nc'
    if not os.path.exists(file) or reload:
        try:
            link = data_info_dict[name]['link']
        except KeyError:
            raise KeyError(f'No item {name} in data_info.json')
        os.system(f"wget -O {file} \"{link}\"")
    else:
        print('file already loaded')

## Working with climate data
We need to understand how the data looks, whether there are some outright anomalies, lest we succumb to the consequences of the principle:
> ``garbage in garbage out''
>
> Used to express the idea that in computing and other spheres, incorrect or poor quality input will always produce faulty output (often abbreviated as GIGO). The saying is recorded from the mid 20th century.

### Plasim data
* Look at some overall statistics, mean, std, etc

    - t2m: two meter temperature
    - zg500: 500 mbar geopotential
    - mrso: soil moisture
    
* plot few times with geocontour and geocontourf that we provide

### Area integrals
* Define the area of France (or some other place)
$$
\alpha(t):=\frac{1}{|\mathcal{D}|} \int_{D}\left(T_{2m}-\mathbb{E}\left(T_{2m}\right)\right)(\vec{r}, u) \mathrm{d} \vec{r} \mathrm{~d} u
$$
* Compute area intregrals
* Look at the time series, statistics, mean, std
* Compute the running mean of T-days (T=14 days)
    $$
        A(t):=\frac{1}{T} \int_{t}^{t+T} \alpha(t)  
    $$

## Define labels 
* Define heat wave labels (Choose 95 percentile of $A(t)$ for example)
---

# Machine Learning

### Probabilistic Prediction
* Familiarize yourself with skill scores
    - Brier score:
        $$ 
            S=\frac{1}{2}\sum_{k=0}^{K-1}\left[\delta_{Y,k}-\hat{p}_{k}(X)\right]^{2}
        $$
    - Logarithmic score
    - Matthew's Correlation Coefficient ($\phi$-coefficient)
* The goal is to predict probabilities

### Simple classifier
Before trying something complex always resort to good-old techniques to see what can be achieved without much effort
* Perform train-validation-test 
* Try undersampling the negative labels to accelerate learning and save some RAM
* Logistic regression, Naive Bayes, kNN, etc
* See what can be achieved with simple persistance
* Try using soil moisture or geopotential in the same spirit

# Neural networks

### Convolutional Neural Network
* Design a simple CNN architecture with a dense layer and sigmoid/softmax output
* Start by providing input with 2-meter temperature, or geopotential (results will vary)
* Normalize the original data (try different ways)
* Put the data through the pipeline and train the network

# Hyperparameter optimization

* Try stacking different fields together
* Implement early stopping to save time

You can try performing grid-search, random-search or bayesian-search using optuna (recommended)

### optimization using `optuna`

In [2]:
import optuna

In [ ]:
def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()

study.optimize(objective, n_trials=100)
